In [1]:
import numpy as np 
import pandas as pd 

## 1.	Read Security Reference input file and convert it to DataFrame using column names supplied in input file: corp_pfd.diff

In [2]:
# importing files 
ref_fields = pd.read_csv('reference_fields.csv')
ref_sec = pd.read_csv('reference_securities.csv')
ref_sec.columns = map(str.upper, ref_sec.columns)

In [3]:
ref_fields.head()

,field,id_field
0,3MTH_IMPVOL_100.0%MNY_DF,0
1,3MTH_IMPVOL_90.0%MNY_DF,0
2,ACCRUED_INTEREST_TO_DATE,0
3,ADR_SH_PER_ADR,0
4,ALTMAN_Z_SCORE,0


In [4]:
# find column names and parse the data 
fields = []
data = []
with open(r"corp_pfd.dif") as f:
    focus = "columns"
    for line in f:
        line = line.strip("# \n")        # clean up whitespace and # for lines
        if not line:                     # skip empty lines
            continue
        if focus == "columns":
            if line == 'END-OF-FIELDS':
                focus = "data"
            if line.isupper():
                fields.append(line)
        if focus == "data":
            if line == 'END-OF-DATA':
                break
            data.append(line)
            
        
fields_new = fields[2:]
data_new = data[3:]

In [5]:
# combine into dataframe
df = pd.DataFrame([sub.split("|") for sub in data_new],columns = fields_new)
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
df.head()

,TICKER_YELLOW_KEY,ST1,REF1,TICKER,CPN,MATURITY,SERIES,NAME,SHORT_NAME,ISSUER_INDUSTRY,...,INDUSTRY_GROUP_NUM,INDUSTRY_SECTOR_NUM,ISSUERS_STOCK,INFLATION_LAG,MAKE_WHOLE_CALL_SPREAD,ISSUER_BULK,ID_BB_SEC_NUM_DES,FEED_SOURCE,ID_BB_GLOBAL_COMPANY,ID_BB_GLOBAL_COMPANY_NAME
0,AS3629704 Corp,-1,210,MTRC,2.560000,20210502,EMTN,MTR CORP CI LTD,MTR CORP CI LTD,SPECIAL PURPOSE,...,20091,10011,,N.A.,N.A.,MTR CORP CI LTD,MTRC 2.56 05/02/21 EMTN,BGN,BBG001GBHFS3,MTR Corp CI Ltd
1,AS3764410 Corp,-1,210,CILBAR,33.921460,20210502,I,CILBRAKE SRL,CILBRAKE SRL,INDUSTRIAL,...,20022,10004,,N.A.,N.A.,CILBRAKE SRL,CILBAR F 05/02/21 I,BGN,BBG00KRC8B42,Cilbrake SRL
2,ZS2774298 Corp,-1,210,REDPRO,9.875000,20210502,,REDCO PROPERTIES GROUP,REDCO PROPERTIES,FINANCIAL,...,20058,10008,,N.A.,N.A.,REDCO PROPERTIES GROUP,REDPRO 9.875 05/02/21,BGN,BBG005WQH7K9,Redco Properties Group Ltd
3,4590568X8 Corp,0,210,IBRD,N.A.,20831201,,INTL BK RECON & DEVELOP,INT BK RECON&DEV,SUPRA-NATIONAL,...,20073,10010,,N.A.,N.A.,INTL BK RECON & DEVELOP,IBRD V0 12/01/83,BGN,BBG001FHC3T2,International Bank for Reconstruction & Develo...
4,AF1220108 Corp,0,210,POSADA,7.875000,20220630,144A,GRUPO POSADAS SAB CV,GRUPO POSADAS,INDUSTRIAL,...,20029,10004,,N.A.,50.000,GRUPO POSADAS SAB CV,POSADA 7.875 06/30/22 144A,BGN,BBG001FGB048,Grupo Posadas SAB de CV


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2896 entries, 0 to 2895
Columns: 213 entries, TICKER_YELLOW_KEY to ID_BB_GLOBAL_COMPANY_NAME
dtypes: object(213)
memory usage: 4.7+ MB


# 2.	Limit columns in DataFrame to only those found in reference_fields.csv

In [7]:
df_limit = df.filter(items= ref_fields['field'].tolist())
df_limit.head()

,AMT_OUTSTANDING,CALC_TYP_DES,CNTRY_OF_DOMICILE,CNTRY_OF_INCORPORATION,COLLAT_TYP,CONTINGENT_CONVERSION,COUNTRY,CPN,CPN_TYP,CRNCY,...,PCT_PAR_QUOTED,REDEMP_VAL,REFERENCE_INDEX,SECURITY_DES,SECURITY_TYP,SECURITY_TYP2,SINKABLE,SINKING_FUND_FACTOR,TICKER,TRADE_CRNCY
0,413000000.00,HONG KONG GOVT BDS,HK,KY,COMPANY GUARNT,,CI,2.560000,FIXED,HKD,...,Y,100.00000,,MTRC 2.56 05/02/21,EURO MTN,,N,1.00000000,MTRC,HKD
1,6666680.00,ARGENT FLT,AR,AR,UNSECURED,,AR,33.921460,FLOATING,ARS,...,Y,100.00000,,CILBAR Float 05/02/21,DOMESTIC,,Y,1.00000000,CILBAR,ARS
2,128524000.00,STREET CONVENTION,CN,KY,SR SECURED,,CI,9.875000,FIXED,USD,...,Y,100.00000,,REDPRO 9 7/8 05/02/21,EURO-DOLLAR,,N,1.00000000,REDPRO,USD
3,200000000.00,NO CALC-FLOATERS,US,SNAT,DEBENTURES,,SNAT,N.A.,VARIABLE,CAD,...,Y,100.00000,,IBRD Float 12/01/83,EURO NON-DOLLAR,,N,1.00000000,IBRD,CAD
4,392605000.00,STREET CONVENTION,MX,MX,COMPANY GUARNT,,MX,7.875000,DEFAULTED,USD,...,Y,100.00000,,POSADA 7 7/8 06/30/22,PRIV PLACEMENT,,N,1.00000000,POSADA,USD


# 3.	Compare securities in input file with reference_securities.csv file.  Use ID_BB_GLOBAL as the unique key.   Create new_securities.csv, which should include all securities NOT found in reference_securities.csv.   new_securities.csv structure should match reference_securities.csv (Note.  if ID_BB_GLOBAL is NOT unique in input file – use first available row with same ID_BB_GLOBAL)

In [8]:
new_columns = ref_sec.columns # get columns from reference_securities

# find rows in df where ID_BB_GLOBAL does not exist in reference_securities

common = df_limit.merge(ref_sec, on=["ID_BB_GLOBAL"]) 

result = df_limit[~df_limit.ID_BB_GLOBAL.isin(common.ID_BB_GLOBAL)][new_columns]

In [9]:
result

,ID_BB_GLOBAL,ID_ISIN,ID_CUSIP,ID_SEDOL1,TICKER,NAME,EXCH_CODE,ISSUER,MARKET_SECTOR_DES
1446,BBG000064N91,SE0001233990,ED5429110,B7NCQ92,OREKON,ORESUNDSKONSORTIET,NOMX STOCKHOLM,ORESUNDSKONSORTIET,Govt


In [10]:
# make csv file 
result.to_csv('new_securities.csv')

# 4.	Create security_data.csv   (comma ‘,’ delimited) with the following structure:

In [11]:
# Create new dataframe with custom columns 
s_data = pd.DataFrame(columns = ['ID_BB_GLOBAL', 'FIELD','VALUE','SOURCE','TSTAMP'])

s_columns = result.columns.values.tolist()
for id_bb in range(len(result)):
    print(id_bb)
    print(s_columns)
    g = result.iloc[id_bb]
    print(g, '\n')
    tstamp = "2021-05-03 19:50:31"
    source = "corp_pfd.dif"
    for column in range(len(g)):
        code = result.iloc[id_bb]['ID_BB_GLOBAL']
        field = s_columns[column]
        value = g[column]
        s_data.loc[len(s_data.index)] = (code ,field, value ,source,tstamp )
    
    


0
['ID_BB_GLOBAL', 'ID_ISIN', 'ID_CUSIP', 'ID_SEDOL1', 'TICKER', 'NAME', 'EXCH_CODE', 'ISSUER', 'MARKET_SECTOR_DES']
ID_BB_GLOBAL               BBG000064N91
ID_ISIN                    SE0001233990
ID_CUSIP                      ED5429110
ID_SEDOL1                       B7NCQ92
TICKER                           OREKON
NAME                 ORESUNDSKONSORTIET
EXCH_CODE                NOMX STOCKHOLM
ISSUER               ORESUNDSKONSORTIET
MARKET_SECTOR_DES                  Govt
Name: 1446, dtype: object 



In [12]:
s_data

,ID_BB_GLOBAL,FIELD,VALUE,SOURCE,TSTAMP
0,BBG000064N91,ID_BB_GLOBAL,BBG000064N91,corp_pfd.dif,2021-05-03 19:50:31
1,BBG000064N91,ID_ISIN,SE0001233990,corp_pfd.dif,2021-05-03 19:50:31
2,BBG000064N91,ID_CUSIP,ED5429110,corp_pfd.dif,2021-05-03 19:50:31
3,BBG000064N91,ID_SEDOL1,B7NCQ92,corp_pfd.dif,2021-05-03 19:50:31
4,BBG000064N91,TICKER,OREKON,corp_pfd.dif,2021-05-03 19:50:31
5,BBG000064N91,NAME,ORESUNDSKONSORTIET,corp_pfd.dif,2021-05-03 19:50:31
6,BBG000064N91,EXCH_CODE,NOMX STOCKHOLM,corp_pfd.dif,2021-05-03 19:50:31
7,BBG000064N91,ISSUER,ORESUNDSKONSORTIET,corp_pfd.dif,2021-05-03 19:50:31
8,BBG000064N91,MARKET_SECTOR_DES,Govt,corp_pfd.dif,2021-05-03 19:50:31


In [13]:
for i in range(3):
    print(data[i])

END-OF-FIELDS
TIMESTARTED=Mon May  3 19:50:31 EDT 2021
START-OF-DATA


In [14]:
# make csv file
s_data.to_csv('security_data.csv', sep =',', index = False)